In [2]:
# pip install lightgbm

     ---------------------------------------- 1.0/1.0 MB 16.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from tensorflow import keras
from keras.models import Sequential
from keras import Input
from keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.callbacks import EarlyStopping

from lightgbm import LGBMRegressor

In [7]:
# epochs = 100
# batch_size = 32
# drop_rate = 0.3

max_depth = 5
n_estimators = 400             
lr = 0.01
num_leaves = 31
early_stopping_rounds = 100     
verbose = True                  

In [8]:
curr_dir = os.getcwd()
curr_dir

'C:\\weather\\v_lgbm'

In [21]:
def model_train(group_number, curr_dir):
 
    path_train = os.path.join(curr_dir+"\\data_group\\"+"group"+str(group_number)+"\\train")
    path_test = os.path.join(curr_dir+"\\data_group\\"+"group"+str(group_number)+"\\test")
    
    folders_tr = os.listdir(path_train)
    folders_test = os.listdir(path_test)
    
    train = pd.DataFrame()
    for files in folders_tr:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_group\\"+"group"+str(group_number)+"\\train\\")+files)
        train = pd.concat([train, df])            
              
    
    test = pd.DataFrame()
    for files in folders_test:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_group\\"+"group"+str(group_number)+"\\test\\")+files)
        test = pd.concat([test, df])   

        
    X_train = train.drop(columns = ['YearMonthDayHourMinute','STN', 'Lon', 'Lat', 'isitu-LST', 'insitu-TA'])
    X_test = test.drop(columns = ['YearMonthDayHourMinute','STN', 'Lon', 'Lat', 'isitu-LST', 'insitu-TA'])
    
    y_train_LST=train[['isitu-LST']].values
    y_test_LST=test[['isitu-LST']].values
    
    y_train_TA=train[['insitu-TA']].values
    y_test_TA=test[['insitu-TA']].values
    
#     model_name = '\\save\\'+'best_model_v1.h5' 

#     filepath = os.path.join(os.getcwd()+ model_name)

#     es = EarlyStopping(monitor='val_mse', verbose=1, patience=20)

#     checkpoint = ModelCheckpoint(filepath=filepath,
#                              monitor='val_mse',
#                              verbose=1,
#                              save_best_only=True)

#     callbacks = [checkpoint, es]

    lgbm_reg_LST = LGBMRegressor(objective= "regression",
                          max_depth= max_depth,
                          n_estimators= n_estimators,
                          learning_rate= lr,
                          num_leaves = num_leaves)
    
    lgbm_reg_LST.fit(X_train, y_train_LST,
              eval_set=[(X_test, y_test_LST)],
              eval_metric=["mse"],
              early_stopping_rounds=early_stopping_rounds,
              verbose=verbose)
    
    lgbm_reg_TA = LGBMRegressor(objective= "regression",
                          max_depth= max_depth,
                          n_estimators= n_estimators,
                          learning_rate= lr,
                          num_leaves = num_leaves)
    
    lgbm_reg_TA.fit(X_train, y_train_TA,
              eval_set=[(X_test, y_test_TA)],
              eval_metric=["mse"],
              early_stopping_rounds=early_stopping_rounds,
              verbose=verbose) 
    
 
    
#     lgbm_reg.load_weights(model_name)
    
    
    path_val = os.path.join(curr_dir+"\\data_group\\"+"group"+str(group_number)+"\\validation")
    folders_val = os.listdir(path_val)
    
    validation = pd.DataFrame()
    for files in folders_val:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_group\\"+"group"+str(group_number)+"\\validation\\")+files)
        validation = pd.concat([validation, df])        
        
    validation_ = validation.drop(columns = ['YearMonthDayHourMinute','STN', 'Lon', 'Lat'])
    prediction_LST = lgbm_reg_LST.predict(validation_)
    prediction_TA = lgbm_reg_TA.predict(validation_)
    
    validation_partial = validation[['YearMonthDayHourMinute', 'STN']]
    validation_partial = validation_partial.assign(LST = prediction_LST)
    validation_partial = validation_partial.assign(TA = prediction_TA)
    validation_partial.to_csv("lgbm_group"+str(group_number)+"_v4.csv", index = False)

In [22]:
for i in tqdm(range(1, 8)):           
    model_train(i, curr_dir)

  0%|                                                                                            | 0/7 [00:00<?, ?it/s]C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-ve

[1]	valid_0's l2: 98.3227
[2]	valid_0's l2: 96.4078
[3]	valid_0's l2: 94.5345
[4]	valid_0's l2: 92.6976
[5]	valid_0's l2: 90.8981
[6]	valid_0's l2: 89.1341
[7]	valid_0's l2: 87.4052
[8]	valid_0's l2: 85.7112
[9]	valid_0's l2: 84.053
[10]	valid_0's l2: 82.4312
[11]	valid_0's l2: 80.8416
[12]	valid_0's l2: 79.2775
[13]	valid_0's l2: 77.7446
[14]	valid_0's l2: 76.2499
[15]	valid_0's l2: 74.7806
[16]	valid_0's l2: 73.3373
[17]	valid_0's l2: 71.9269
[18]	valid_0's l2: 70.5541
[19]	valid_0's l2: 69.1974
[20]	valid_0's l2: 67.8878
[21]	valid_0's l2: 66.5875
[22]	valid_0's l2: 65.3186
[23]	valid_0's l2: 64.0654
[24]	valid_0's l2: 62.8473
[25]	valid_0's l2: 61.6573
[26]	valid_0's l2: 60.4891
[27]	valid_0's l2: 59.3351
[28]	valid_0's l2: 58.2081
[29]	valid_0's l2: 57.1017
[30]	valid_0's l2: 56.0139
[31]	valid_0's l2: 54.963
[32]	valid_0's l2: 53.928
[33]	valid_0's l2: 52.9061
[34]	valid_0's l2: 51.9147
[35]	valid_0's l2: 50.9417
[36]	valid_0's l2: 49.9789
[37]	valid_0's l2: 49.044
[38]	valid_0's

[313]	valid_0's l2: 3.48972
[314]	valid_0's l2: 3.48649
[315]	valid_0's l2: 3.48381
[316]	valid_0's l2: 3.48051
[317]	valid_0's l2: 3.47741
[318]	valid_0's l2: 3.4738
[319]	valid_0's l2: 3.47042
[320]	valid_0's l2: 3.46721
[321]	valid_0's l2: 3.46358
[322]	valid_0's l2: 3.46092
[323]	valid_0's l2: 3.45551
[324]	valid_0's l2: 3.45321
[325]	valid_0's l2: 3.4503
[326]	valid_0's l2: 3.4456
[327]	valid_0's l2: 3.44345
[328]	valid_0's l2: 3.44203
[329]	valid_0's l2: 3.43958
[330]	valid_0's l2: 3.43733
[331]	valid_0's l2: 3.43481
[332]	valid_0's l2: 3.43358
[333]	valid_0's l2: 3.43152
[334]	valid_0's l2: 3.42985
[335]	valid_0's l2: 3.42851
[336]	valid_0's l2: 3.42657
[337]	valid_0's l2: 3.42252
[338]	valid_0's l2: 3.42191
[339]	valid_0's l2: 3.42063
[340]	valid_0's l2: 3.41955
[341]	valid_0's l2: 3.41783
[342]	valid_0's l2: 3.41615
[343]	valid_0's l2: 3.41455
[344]	valid_0's l2: 3.41282
[345]	valid_0's l2: 3.40902
[346]	valid_0's l2: 3.407
[347]	valid_0's l2: 3.40541
[348]	valid_0's l2: 3.403

[218]	valid_0's l2: 3.31247
[219]	valid_0's l2: 3.28268
[220]	valid_0's l2: 3.25363
[221]	valid_0's l2: 3.22474
[222]	valid_0's l2: 3.19662
[223]	valid_0's l2: 3.16907
[224]	valid_0's l2: 3.1409
[225]	valid_0's l2: 3.11427
[226]	valid_0's l2: 3.08753
[227]	valid_0's l2: 3.062
[228]	valid_0's l2: 3.03514
[229]	valid_0's l2: 3.01069
[230]	valid_0's l2: 2.98617
[231]	valid_0's l2: 2.96104
[232]	valid_0's l2: 2.93779
[233]	valid_0's l2: 2.91467
[234]	valid_0's l2: 2.89119
[235]	valid_0's l2: 2.86879
[236]	valid_0's l2: 2.84673
[237]	valid_0's l2: 2.82414
[238]	valid_0's l2: 2.80277
[239]	valid_0's l2: 2.78069
[240]	valid_0's l2: 2.76013
[241]	valid_0's l2: 2.73963
[242]	valid_0's l2: 2.71979
[243]	valid_0's l2: 2.69921
[244]	valid_0's l2: 2.67963
[245]	valid_0's l2: 2.65953
[246]	valid_0's l2: 2.6407
[247]	valid_0's l2: 2.62131
[248]	valid_0's l2: 2.6028
[249]	valid_0's l2: 2.58551
[250]	valid_0's l2: 2.56701
[251]	valid_0's l2: 2.54894
[252]	valid_0's l2: 2.53218
[253]	valid_0's l2: 2.514

 14%|████████████                                                                        | 1/7 [00:17<01:47, 17.85s/it]

...

[1]	valid_0's l2: 100.14
[2]	valid_0's l2: 98.361
[3]	valid_0's l2: 96.6227
[4]	valid_0's l2: 94.9171
[5]	valid_0's l2: 93.2521
[6]	valid_0's l2: 91.6092
[7]	valid_0's l2: 89.987
[8]	valid_0's l2: 88.4248
[9]	valid_0's l2: 86.8379
[10]	valid_0's l2: 85.3118
[11]	valid_0's l2: 83.7834
[12]	valid_0's l2: 82.3456
[13]	valid_0's l2: 80.9053
[14]	valid_0's l2: 79.4761
[15]	valid_0's l2: 78.0845
[16]	valid_0's l2: 76.7647
[17]	valid_0's l2: 75.4273
[18]	valid_0's l2: 74.1406
[19]	valid_0's l2: 72.8685
[20]	valid_0's l2: 71.6444
[21]	valid_0's l2: 70.4183
[22]	valid_0's l2: 69.1723
[23]	valid_0's l2: 68.0268
[24]	valid_0's l2: 66.8484
[25]	valid_0's l2: 65.7359
[26]	valid_0's l2: 64.6387
[27]	valid_0's l2: 63.5641
[28]	valid_0's l2: 62.5238
[29]	valid_0's l2: 61.4531
[30]	valid_0's l2: 60.4358
[31]	valid_0's l2: 59.4539
[32]	valid_0's l2: 58.4844
[33]	valid_0's l2: 57.5325
[34]	valid_0's l2: 56.6378
[35]	valid_0's l2: 55.7298
[36]	valid_0's l2: 54.8444
[37]	valid_0's l2: 53.9651
[38]	valid_0'

[1]	valid_0's l2: 71.5861
[2]	valid_0's l2: 70.2517
[3]	valid_0's l2: 68.9455
[4]	valid_0's l2: 67.6613
[5]	valid_0's l2: 66.401
[6]	valid_0's l2: 65.1698
[7]	valid_0's l2: 63.9571
[8]	valid_0's l2: 62.7754
[9]	valid_0's l2: 61.5801
[10]	valid_0's l2: 60.4347
[11]	valid_0's l2: 59.2799
[12]	valid_0's l2: 58.186
[13]	valid_0's l2: 57.0815
[14]	valid_0's l2: 56.0005
[15]	valid_0's l2: 54.9404
[16]	valid_0's l2: 53.9059
[17]	valid_0's l2: 52.8943
[18]	valid_0's l2: 51.9016
[19]	valid_0's l2: 50.9051
[20]	valid_0's l2: 49.9581
[21]	valid_0's l2: 49.0116
[22]	valid_0's l2: 48.1098
[23]	valid_0's l2: 47.2157
[24]	valid_0's l2: 46.3072
[25]	valid_0's l2: 45.4209
[26]	valid_0's l2: 44.5824
[27]	valid_0's l2: 43.7252
[28]	valid_0's l2: 42.9178
[29]	valid_0's l2: 42.1254
[30]	valid_0's l2: 41.316
[31]	valid_0's l2: 40.5267
[32]	valid_0's l2: 39.7805
[33]	valid_0's l2: 39.0166
[34]	valid_0's l2: 38.3173
[35]	valid_0's l2: 37.5729
[36]	valid_0's l2: 36.8862
[37]	valid_0's l2: 36.1834
[38]	valid_0'

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:58<00:00, 15.49s/it]

[217]	valid_0's l2: 1.48984
[218]	valid_0's l2: 1.46661
[219]	valid_0's l2: 1.44441
[220]	valid_0's l2: 1.42222
[221]	valid_0's l2: 1.4003
[222]	valid_0's l2: 1.37881
[223]	valid_0's l2: 1.35769
[224]	valid_0's l2: 1.33781
[225]	valid_0's l2: 1.31748
[226]	valid_0's l2: 1.29674
[227]	valid_0's l2: 1.27687
[228]	valid_0's l2: 1.25799
[229]	valid_0's l2: 1.23908
[230]	valid_0's l2: 1.22055
[231]	valid_0's l2: 1.20324
[232]	valid_0's l2: 1.1849
[233]	valid_0's l2: 1.16764
[234]	valid_0's l2: 1.1503
[235]	valid_0's l2: 1.13323
[236]	valid_0's l2: 1.1162
[237]	valid_0's l2: 1.09915
[238]	valid_0's l2: 1.08357
[239]	valid_0's l2: 1.06753
[240]	valid_0's l2: 1.05193
[241]	valid_0's l2: 1.03721
[242]	valid_0's l2: 1.02171
[243]	valid_0's l2: 1.00631
[244]	valid_0's l2: 0.992288
[245]	valid_0's l2: 0.977636
[246]	valid_0's l2: 0.963369
[247]	valid_0's l2: 0.94995
[248]	valid_0's l2: 0.935919
[249]	valid_0's l2: 0.922621
[250]	valid_0's l2: 0.909687
[251]	valid_0's l2: 0.896919
[252]	valid_0's l

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:58<00:00, 16.97s/it]


# submission

In [26]:
submission=pd.read_csv('1-2_검증데이터셋.csv', encoding='utf-8')
submission1=submission.copy()
submission1

submission1['index']=submission1.index
submission1 = submission1.sort_values(['STN'])
submission1

,YearMonthDayHourMinute,STN,isitu-LST,insitu-TA,index
0,202107010030,90,NaN,NaN,0
32867,202108230530,90,NaN,NaN,32867
19103,202107081630,90,NaN,NaN,19103
32970,202108230630,90,NaN,NaN,32970
51330,202108301730,90,NaN,NaN,51330
...,...,...,...,...,...
41077,202108261230,980,NaN,NaN,41077
40975,202108261130,980,NaN,NaN,40975
40873,202108261030,980,NaN,NaN,40873
41380,202108261530,980,NaN,NaN,41380


In [27]:
group = []
for i in range(1,8):
    group.append(pd.read_csv("lgbm_group"+str(i)+"_v4.csv"))
group

[      YearMonthDayHourMinute  STN        LST         TA
 0               2.021070e+11   93  21.198707  20.311800
 1               2.021070e+11  101  21.838729  21.112782
 2               2.021070e+11  104  20.638057  19.968788
 3               2.021070e+11  105  21.658366  21.409213
 4               2.021070e+11  108  23.019507  22.469618
 ...                      ...  ...        ...        ...
 7379            2.021080e+11  138  21.476546  20.701416
 7380            2.021080e+11  159  24.902596  24.674699
 7381            2.021080e+11  165  25.416498  25.837450
 7382            2.021080e+11  168  26.319507  26.456392
 7383            2.021080e+11  184  26.424280  26.575623
 
 [7384 rows x 4 columns],
       YearMonthDayHourMinute  STN        LST         TA
 0               2.021070e+11  702  22.916391  22.293156
 1               2.021070e+11  970  20.436796  19.461710
 2               2.021070e+11  977  23.014647  21.510056
 3               2.021070e+11  978  21.445577  19.865945
 4 

In [34]:
concat = []
for i in range(len(group)):
    group[i] = group[i].sort_values(['STN'])
    a=list(set(group[i]['STN'].values))
    a.sort()
    
    tr=pd.DataFrame()
    for j in range(len(a)):
        sub = submission[submission['STN']== a[j]]
        tr = pd.concat([tr,sub])
    tr['index'] = tr.index   
    tr[['isitu-LST', 'insitu-TA']] = group[i][['LST', 'TA']].values
    
    concat.append(tr)

In [38]:
submission2 = pd.DataFrame()
for files in concat:
    submission2 = pd.concat([submission2, files])      
submission2

,YearMonthDayHourMinute,STN,isitu-LST,insitu-TA,index
1,202107010030,93,21.198707,20.311800,1
104,202107010130,93,19.574467,19.366177,104
207,202107010230,93,27.074587,23.711782,207
310,202107010330,93,24.500026,23.110872,310
413,202107010430,93,23.690163,22.677879,413
...,...,...,...,...,...
54014,202108311930,140,25.962752,26.755826,54014
54117,202108312030,140,25.020140,25.162566,54117
54220,202108312130,140,25.702848,25.147550,54220
54323,202108312230,140,30.122997,25.428211,54323


In [40]:
sub = submission2.sort_values(by='index')
sub = sub.drop('index', axis=1)
sub

,YearMonthDayHourMinute,STN,isitu-LST,insitu-TA
0,202107010030,90,21.562070,20.843918
1,202107010030,93,21.198707,20.311800
2,202107010030,95,25.535438,23.915842
3,202107010030,98,32.609842,24.803199
4,202107010030,99,22.167850,21.190524
...,...,...,...,...
54498,202108312330,973,25.143085,23.934039
54499,202108312330,974,21.924462,23.649829
54500,202108312330,977,24.281951,22.891516
54501,202108312330,978,21.333833,19.821098


In [42]:
sub.to_csv('lgbm_v4.csv', index = False)